In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd

import sys, os
import sklearn
import datetime

import importlib
from tqdm import tqdm

In [4]:
import st_toolkit as geohl
importlib.reload(geohl)

import cri_calc as cri
importlib.reload(cri)

import cri_helper as helper
importlib.reload(helper)

In [7]:
def calculate_cri(rec_own, rec_target):
    own = rec_own._asdict()
    target = rec_target._asdict()
    
    ves_dcpa, ves_tcpa, hr, rel_movement_angle, dist_euclid, speed_r = cri.colregs_alarms(own=own, target=target)
    ves_cri = cri.calculate_cri(own, target, ves_dcpa, ves_tcpa, hr, rel_movement_angle, dist_euclid, speed_r)
    
    return ves_dcpa, ves_tcpa, hr, rel_movement_angle, dist_euclid, speed_r, ves_cri

In [8]:
def calculate_cri_timeslice(df):
    timeslice_result = []
    
    for row_i in df.itertuples():
        for row_j in df.itertuples():
            if row_i.Index == row_j.Index:
                continue
                
            timeslice_result.append([row_i.Index, row_i.mmsi, row_i.geom, row_i.speed, row_i.course, 
                                     row_j.Index, row_j.mmsi, row_j.geom, row_j.speed, row_j.course, *calculate_cri(row_i, row_j)])
            
#     return pd.DataFrame(timeslice_result, columns=['own', 'target', 'dcpa', 'tcpa', 'hr', 'rel_movement_angle', 'dist_euclid', 'speed_r', 'cri'])
    return pd.DataFrame(timeslice_result, columns=['own_Index', 'own_mmsi', 'own_geom', 'own_speed', 'own_course',
                                                   'target_Index', 'target_mmsi', 'target_geom', 'target_speed', 'target_course', 
                                                   'dcpa', 'tcpa', 'hr', 'rel_movement_angle', 'dist_euclid', 'speed_r', 'cri'])

In [49]:
def ml_calc_cri(rec_own, rec_target, model=None, model_fun=calculate_cri, model_norm=None):
    own = rec_own
    target = rec_target
    
    if model is None:
        _, _, _, _, dist_euclid, _, ves_cri = model_fun(own, target)
    else:
        dist_euclid, model_input = model_fun(own, target)
        ves_cri = model.predict(model_norm.transform(np.array(model_input).reshape(1, -1)))
    
    return dist_euclid, min(max(ves_cri[0], 0), 1)

In [42]:
def ml_calc_cri_timeslice(df, **kwargs):
    timeslice_result = []
    
    for row_i in df.itertuples():
        for row_j in df.itertuples():
            if row_i.Index == row_j.Index:
                continue
                
            timeslice_result.append([row_i.Index, row_i.mmsi, row_i.geom, row_i.speed, row_i.course, 
                                     row_j.Index, row_j.mmsi, row_j.geom, row_j.speed, row_j.course, *ml_calc_cri(row_i, row_j, **kwargs)])
            
#     return pd.DataFrame(timeslice_result, columns=['own', 'target', 'dcpa', 'tcpa', 'hr', 'rel_movement_angle', 'dist_euclid', 'speed_r', 'cri'])
    return pd.DataFrame(timeslice_result, columns=['own_Index', 'own_mmsi', 'own_geom', 'own_speed', 'own_course',
                                                   'target_Index', 'target_mmsi', 'target_geom', 'target_speed', 'target_course', 
                                                   'dist_euclid', 'cri'])

# Load Data

In [9]:
df = pd.read_csv('./data/unipi_ais_dynamic_jul2018_1w_algn_linear_v2_w_lens.csv', parse_dates=['datetime'])
gdf = geohl.getGeoDataFrame_v2(df, crs='epsg:4326')

/home/andrewt/miniconda3/envs/vesai/lib/python3.8/site-packages/pandas/core/dtypes/cast.py:122: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


In [10]:
gdf2 = gdf.loc[gdf.datetime.dt.date.between(datetime.date(2018, 7, 3), datetime.date(2018, 7, 3), inclusive='both')].copy()

In [11]:
gdf_sub_moving = gdf2.loc[gdf2.speed.between(1, 50, inclusive='neither')].copy()

In [12]:
gdf_vcra = pd.read_pickle('./data/unipi_ais_dynamic_jul2018_1w_vcra_dataset_v3.pickle')

In [13]:
tqdm.pandas(desc='Adding Vessels\' Length...')

# gdf_vcra.loc[:, 'own_length'] = gdf_vcra.own_Index.apply(lambda l: gdf_sub_moving[l].length)
mlp_input = gdf_vcra.loc[gdf_vcra.own_Index.isin(gdf_sub_moving.index.values)].copy()
mlp_input.loc[:, 'own_length'] = mlp_input.own_Index.progress_apply(lambda l: gdf_sub_moving.loc[l].length)
mlp_input.loc[:, 'target_length'] = mlp_input.target_Index.progress_apply(lambda l: gdf_sub_moving.loc[l].length)

Adding Vessels' Length...: 100%|██████| 960268/960268 [01:37<00:00, 9865.41it/s]


In [21]:
grouped = gdf_sub_moving.groupby(['datetime'])
l = grouped.get_group((list(grouped.groups)[0]))

## Evaluating EQ model timeliness

In [32]:
def calculate_cri(rec_own, rec_target):
    own = rec_own._asdict()
    target = rec_target._asdict()
    
    ves_dcpa, ves_tcpa, hr, rel_movement_angle, dist_euclid, speed_r = cri.colregs_alarms(own=own, target=target)
    ves_cri = cri.calculate_cri(own, target, ves_dcpa, ves_tcpa, hr, rel_movement_angle, dist_euclid, speed_r)
    
    return ves_dcpa, ves_tcpa, hr, rel_movement_angle, dist_euclid, speed_r, ves_cri

In [34]:
%%timeit 
ml_calc_cri_timeslice(l.copy(), model=None, model_fun=calculate_cri, model_norm=None);

329 ms ± 11.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Compare with Park et al.

In [14]:
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import mean_absolute_error, mean_squared_error

from skrvm import RVR

In [15]:
X = mlp_input[['dist_euclid', 'hr', 'own_speed', 'target_speed', 'own_course', 'target_course', 'own_length', 'target_length']].copy()
X = X.values
y = mlp_input[['cri']].values.ravel()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

In [ ]:
n_samples = 15000

scaler = StandardScaler()
X_train_norm = scaler.fit_transform(X_train)


clf = RVR(kernel='rbf', verbose=False, n_iter=100)
clf.fit(X_train_norm[:n_samples].astype(float), y_train[:n_samples].astype(float))

clf.score(scaler.transform(X_test[:n_samples]), y_test[:n_samples])

In [18]:
cri_pred_rvm = pd.Series(clf.predict(scaler.transform(X_test[:n_samples]))).clip(0,1).values
print(f'MAE: {mean_absolute_error(y_test[:n_samples], cri_pred_rvm)}')
print(f'RMSE: {mean_squared_error(y_test[:n_samples], cri_pred_rvm, squared=False)}')

MAE: 0.03589468107012043
RMSE: 0.08018135415141722


In [19]:
from joblib import dump, load
dump(clf, './data/park-et-al-rvm-vcra-v2.joblib') 

['./data/park-et-al-rvm-vcra-v2.joblib']

In [39]:
def ml_calc_cri_park_etal(rec_own, rec_target):
    own = rec_own._asdict()
    target = rec_target._asdict()
    
    own_geom_nm, target_geom_nm = map(helper.angular_to_nautical_miles, [own['geom'], target['geom']])
    xr, yr = helper.calculate_delta(own_geom_nm.x, target_geom_nm.x), helper.calculate_delta(own_geom_nm.y, target_geom_nm.y)
    hr = helper.calculate_delta(own['course'], target['course'])
    
    # Get vessels' Euclidean Distance -- NAUTICAL MILES
    dist_euclid = np.sqrt(xr**2 + yr**2)
    
    return dist_euclid, [dist_euclid, hr, own['speed'], target['speed'], own['course'], target['course'], own['length'], target['length']]

In [45]:
%%timeit 
ml_calc_cri_timeslice(l.copy(), model=clf, model_fun=ml_calc_cri_park_etal, model_norm=scaler);

322 ms ± 744 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Compare with Li et al.

In [ ]:
from sklearn.preprocessing import MinMaxScaler

X = mlp_input[['speed_r', 'hr', 'rel_movement_angle', 'dist_euclid']].values
y = mlp_input[['cri']].values.ravel()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

scaler = MinMaxScaler()
X_train_norm = scaler.fit_transform(X_train)

regr_li_et_al = MLPRegressor(random_state=10, max_iter=100, hidden_layer_sizes=(54,), 
                    verbose=True, early_stopping=True, n_iter_no_change=10).fit(X_train_norm, y_train)

regr_li_et_al.score(scaler.transform(X_test), y_test)

In [52]:
cri_pred_mlp_li = pd.Series(regr_li_et_al.predict(scaler.transform(X_test))).clip(0,1).values
print(f'MAE: {mean_absolute_error(y_test, cri_pred_mlp_li)}')
print(f'RMSE: {mean_squared_error(y_test, cri_pred_mlp_li, squared=False)}')

MAE: 0.04764371186450534
RMSE: 0.0934240406219107


In [56]:
from joblib import dump, load
dump(regr_li_et_al, './data/li-et-al-mlp-vcra-v2.joblib') 

['./data/li-et-al-mlp-vcra-v2.joblib']

In [54]:
def ml_calc_cri_li_etal(rec_own, rec_target):
    own = rec_own._asdict()
    target = rec_target._asdict()
    
    ves_dcpa, ves_tcpa, hr, rel_movement_angle, dist_euclid, speed_r = cri.colregs_alarms(own=own, target=target)
    
    return dist_euclid, [speed_r, hr, rel_movement_angle, dist_euclid]

In [57]:
%%timeit 
ml_calc_cri_timeslice(l.copy(), model=regr_li_et_al, model_fun=ml_calc_cri_li_etal, model_norm=scaler);

314 ms ± 2.16 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Compare with Gang et al.

In [58]:
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [ ]:
X = mlp_input[['own_course', 'target_course', 'own_speed', 'target_speed', 'hr', 'dist_euclid']].values
y = mlp_input[['cri']].values.ravel()

n_samples = 100000
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

scaler = StandardScaler()
X_train_norm = scaler.fit_transform(X_train[:n_samples])

regr_gang_et_al = SVR(verbose=True).fit(X_train_norm, y_train[:n_samples])

regr_gang_et_al.score(scaler.transform(X_test[:n_samples]), y_test[:n_samples])

In [60]:
cri_pred_svm_gang = pd.Series(regr_gang_et_al.predict(scaler.transform(X_test[:n_samples]))).clip(0,1).values
print(f'MAE: {mean_absolute_error(y_test[:n_samples], cri_pred_svm_gang)}')
print(f'RMSE: {mean_squared_error(y_test[:n_samples], cri_pred_svm_gang, squared=False)}')

MAE: 0.057229761297088434
RMSE: 0.09454195065998014


In [61]:
from joblib import dump, load
dump(regr_gang_et_al, './data/gang-et-al-svm-vcra-v2.joblib') 

['./data/gang-et-al-svm-vcra-v2.joblib']

In [62]:
def ml_calc_cri_gang_etal(rec_own, rec_target):
    own = rec_own._asdict()
    target = rec_target._asdict()
    
    own_geom_nm, target_geom_nm = map(helper.angular_to_nautical_miles, [own['geom'], target['geom']])
    xr, yr = helper.calculate_delta(own_geom_nm.x, target_geom_nm.x), helper.calculate_delta(own_geom_nm.y, target_geom_nm.y)
    hr = helper.calculate_delta(own['course'], target['course'])
    
    # Get vessels' Euclidean Distance -- NAUTICAL MILES
    dist_euclid = np.sqrt(xr**2 + yr**2)
    
    return dist_euclid, [own['course'], target['course'], own['speed'], target['speed'], hr, dist_euclid]

In [63]:
%%timeit 
ml_calc_cri_timeslice(l.copy(), model=regr_gang_et_al, model_fun=ml_calc_cri_gang_etal, model_norm=scaler);

351 ms ± 1.45 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
